<a href="https://colab.research.google.com/github/sda96/AIFFEL_3rd_hackerton_TUNiB_DKTC/blob/main/notebook/SeungTaek/json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd # pandas 모듈 로드
import json  # json 모듈 로드
import re
import os
from glob import glob
from tqdm import tqdm

In [2]:
!gdown https://drive.google.com/uc?id=15NHQXINo1xILHiBvnCWf1XqCtrxlevtx
!gdown https://drive.google.com/uc?id=1dvjlJlEl3VQ4mhd9CybqeEs2Uw1wzqW_

Downloading...
From: https://drive.google.com/uc?id=15NHQXINo1xILHiBvnCWf1XqCtrxlevtx
To: /content/SNS_train.zip
100% 650M/650M [00:12<00:00, 51.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dvjlJlEl3VQ4mhd9CybqeEs2Uw1wzqW_
To: /content/SNS_valid.zip
100% 81.3M/81.3M [00:01<00:00, 43.5MB/s]


In [3]:
!unzip -uq "/content/SNS_train.zip" -d "/content/sns_data"
!unzip -uq "/content/SNS_valid.zip" -d "/content/sns_data"

In [4]:
json_file_path="/content/[라벨]한국어SNS_train/개인및관계.json"
data_path = "/content/sns_data"
files = glob(data_path + "/*/*")
files

['/content/sns_data/[라벨]한국어SNS_train/주거와생활.json',
 '/content/sns_data/[라벨]한국어SNS_train/식음료.json',
 '/content/sns_data/[라벨]한국어SNS_train/여가생활.json',
 '/content/sns_data/[라벨]한국어SNS_train/개인및관계.json',
 '/content/sns_data/[라벨]한국어SNS_train/미용과건강.json',
 '/content/sns_data/[라벨]한국어SNS_train/시사교육.json',
 '/content/sns_data/[라벨]한국어SNS_train/상거래(쇼핑).json',
 '/content/sns_data/[라벨]한국어SNS_train/일과직업.json',
 '/content/sns_data/[라벨]한국어SNS_train/행사.json',
 '/content/sns_data/[라벨]한국어SNS_valid/주거와생활.json',
 '/content/sns_data/[라벨]한국어SNS_valid/식음료.json',
 '/content/sns_data/[라벨]한국어SNS_valid/여가생활.json',
 '/content/sns_data/[라벨]한국어SNS_valid/개인및관계.json',
 '/content/sns_data/[라벨]한국어SNS_valid/미용과건강.json',
 '/content/sns_data/[라벨]한국어SNS_valid/시사교육.json',
 '/content/sns_data/[라벨]한국어SNS_valid/상거래(쇼핑).json',
 '/content/sns_data/[라벨]한국어SNS_valid/일과직업.json',
 '/content/sns_data/[라벨]한국어SNS_valid/행사.json']

In [120]:
files[17:18]

['/content/sns_data/[라벨]한국어SNS_valid/행사.json']

In [45]:
def preprocess_sentence(sentence):

    sentence = re.sub(r"[^0-9]+", " ", sentence)
    sentence = sentence.strip()
    sentence = sentence.zfill(2)
    return sentence

In [121]:
bucket_text = []
for json_file in tqdm(files[17:18]):
    with open(json_file, "r") as f:
        data_dict = json.load(f)
        data_dict = data_dict["data"]
        sentence = []
        for j in data_dict:
            df = pd.json_normalize(j["body"])
            a = []
            for i in range(len(df)):
                b = ''
                if i == 0:
                    b += df['utterance'][i]
                else:
                    if df['turnID'][i] == df['turnID'][i-1]:
                        b += ' ' + df['utterance'][i]
                    else:
                        b += '\n' + df['utterance'][i]
                a.append(b)
            sentence.append(''.join(a))

    bucket_text.append(sentence)

100%|██████████| 1/1 [00:30<00:00, 30.30s/it]


In [122]:
df = pd.DataFrame(bucket_text)
df = df.T
df.head()

,0
0,#@시스템#사진# 죽일까.... 버스타고 징승포가야하나..\n낼갈래? 아님갔다올래\...
1,아냐 한번먹엇을걸 그리고나서 여기보다 남구로가맛잇다고 계속 거기로만갔을거야 근데 뭐...
2,토 밤 가능 금 밤 가능이랭\n금밤은 아주저냑 열시될듯\n허버허버\n저녁약속잇삼\n...
3,오전비행기가 싹 없어 일욜이라서 그런건가? 근데 #@이름# 너도 갈거야?\n갈걸여ㅋ...
4,여름에 예매하쟈\nㅎㅎ아 진짜 알바 해야겟다 다음 방학\n퓨ㅠㅠㅠㅠㅠ나두...\n너...


In [78]:
#!mkdir /content/sns_csv_val

In [123]:
df.to_csv("/content/sns_csv_val/행사.csv", header = True, index = False)

In [124]:
df2 = pd.read_csv("/content/sns_csv_val/행사.csv")
df2.head()

,0
0,#@시스템#사진# 죽일까.... 버스타고 징승포가야하나..\n낼갈래? 아님갔다올래\...
1,아냐 한번먹엇을걸 그리고나서 여기보다 남구로가맛잇다고 계속 거기로만갔을거야 근데 뭐...
2,토 밤 가능 금 밤 가능이랭\n금밤은 아주저냑 열시될듯\n허버허버\n저녁약속잇삼\n...
3,오전비행기가 싹 없어 일욜이라서 그런건가? 근데 #@이름# 너도 갈거야?\n갈걸여ㅋ...
4,여름에 예매하쟈\nㅎㅎ아 진짜 알바 해야겟다 다음 방학\n퓨ㅠㅠㅠㅠㅠ나두...\n너...


In [129]:
!zip -r /content/sns_train.zip /content/sns_csv_train/
!zip -r /content/sns_val.zip /content/sns_csv_val/

  adding: content/sns_csv_train/ (stored 0%)
  adding: content/sns_csv_train/상거래(쇼핑).csv (deflated 60%)
  adding: content/sns_csv_train/주거와생활.csv (deflated 59%)
  adding: content/sns_csv_train/.ipynb_checkpoints/ (stored 0%)
  adding: content/sns_csv_train/미용과건강.csv (deflated 60%)
  adding: content/sns_csv_train/일과직업.csv (deflated 61%)
  adding: content/sns_csv_train/행사.csv (deflated 60%)
  adding: content/sns_csv_train/식음료.csv (deflated 61%)
  adding: content/sns_csv_train/시사교육.csv (deflated 60%)
  adding: content/sns_csv_train/개인및관계.csv (deflated 61%)
  adding: content/sns_csv_train/여가생활.csv (deflated 60%)
  adding: content/sns_csv_val/ (stored 0%)
  adding: content/sns_csv_val/상거래(쇼핑).csv (deflated 60%)
  adding: content/sns_csv_val/주거와생활.csv (deflated 59%)
  adding: content/sns_csv_val/미용과건강.csv (deflated 60%)
  adding: content/sns_csv_val/일과직업.csv (deflated 61%)
  adding: content/sns_csv_val/행사.csv (deflated 60%)
  adding: content/sns_csv_val/식음료.csv (deflated 61%)
  adding: conte

In [130]:
from google.colab import files

files.download('/content/sns_train.zip')
files.download('/content/sns_val.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>